# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
print(df.head(20))
print()
print('________________________________________________________________')
print()
print(df.info())
print()
print('________________________________________________________________')
print()
print(df.isnull().sum())
print()
print('________________________________________________________________')
print()
print(df.describe())
print()
print('________________________________________________________________')
print()
print('-----------------------Тип занятости----------------------------')
print(df['income_type'].unique())
print('------------------------Образование-----------------------------')
print(df['education'].unique())
print('----------------------ID образования----------------------------')
print(df['education_id'].unique())
print('--------------------Семейное положение--------------------------')
print(df['family_status'].unique())
print('------------------ID семейное положение-------------------------')
print(df['family_status_id'].unique())
print('----------------------------Пол---------------------------------')
print(df['gender'].unique())
print(df['gender'].value_counts())
print('-----------------------Тип занятости----------------------------')
print(df['income_type'].unique())
print('----------------------------Долг--------------------------------')
print(df['debt'].unique())
print('------------------------Цели кредита----------------------------')
print(df['purpose'].unique())
print('---------------------------Возраст------------------------------')
print(df['dob_years'].value_counts())

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

### Вывод

Изучил первые 20 строк таблицы. Вызвал метод info()
Обнаружил:
-в столбце 'days_employed' ошибки в виде знаков '-' перед значением,
 а так же отсутствующе значение NaN
 столбец состоит не из целых чисел что может привести к неточностям при анализе
-в столбце 'education' текст написан в разном регистре
-в столбце 'total_income' и 'days_employed' одинаковое число значений
-в интересующих нас столбцах('children' и 'family_status') пропусков нет


### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(df.isnull().sum())
df_group_income = df.groupby('income_type')['days_employed'].mean()
df_group_income[[0,4]] = df_group_income[[0,4]] / 24 
df_group_income = df_group_income / 365
def days(element):
    element / 365 
    if element == 'пенсионер':
        return element / 24 / 365
    if element == 'безработный':
        return element / 24 / 365
    return element / 365

df ['days_employed'] = df['days_employed'].apply(days)
print(df ['days_employed'])

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
0        -23.116912
1        -11.026860
2        -15.406637
3        -11.300677
4        932.235814
            ...    
21520    -12.409087
21521    942.294258
21522     -5.789991
21523     -8.527347
21524     -5.437007
Name: days_employed, Length: 21525, dtype: float64


In [4]:
med_salary = df.groupby('income_type')['total_income'].transform('median')
df['total_income'] = df['total_income'].fillna(med_salary)

med_salary_days = df.groupby('income_type')['days_employed'].transform('median')
df['days_employed'] = df['days_employed'].fillna(med_salary_days)

df['dob_years'] = df['dob_years'].replace(0, df['dob_years'].median())

In [6]:
df[['days_employed', 'children']] = df[['days_employed','children']].abs()

In [7]:
df['children'] = df['children'].replace(20, 2)

In [8]:
print(df['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In [9]:
print(df['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

# Вывод 
1) с помощю мметода insnull проверил таблицу на наличие пустых значений
2) сгрупировал столбец 'days_employed' по 'income_type'. значения в столбце 'days_employed' перевел в года. значения равные 'пенсионер' и 'безработный' поделил на 24 так как посчитал что оно указанно в чисах
3) с помощю метода abc() перевел все числа в их модуль тем самым избавился от отрицательных чисел в столбцах 'days_employed','children'
4) с помощю метода replace() изменил згачение в столбце 'children'  на 2 так как считаю что значение 20 произашло в виду опечатки и дописания лишнего нуля
5)заполнил пустые значения в столбцах 'days_employed','children' по категории 'income_type'. атак же в столбце 'dob_years' с помощю метода replace()

In [12]:
df['gedner'] = df['gender'].replace('XNA', 'F')

### Замена типа данных

In [446]:
df[['days_employed','children','dob_years','education_id','family_status_id','debt','total_income']] = df[['days_employed','children','dob_years','education_id','family_status_id','debt','total_income']].astype('int')

In [447]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод
1) с помощю метода astype() перевел тип данных для болие удобного подсчета и хранения  

### Обработка дубликатов

In [448]:
print(print(df.duplicated().sum()))

55
None


In [449]:
df['education'] = df['education'].str.lower()

In [450]:
print(print(df.duplicated().sum()))

72
None


In [451]:
df.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,12,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21449,0,942,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21450,1,5,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21451,3,8,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

1) в столбце 'education' перевел все значения в нижний регистр для удобства групировки и облегчения выявления дублекатов
2) с помощю метода duplecated() и метода sum() определил колличество грубых повторений
3) с помощью df.drop_duplicates().reset_index(drop=True) избавился от повторений удалением строк с повторением и выравнял их индекс

### Лемматизация

In [452]:
from pymystem3 import Mystem
m = Mystem()
def lemma(queries):
    queries_list = m.lemmatize(queries)
    if 'образование' in queries_list:
        return 'образование'
    if 'свадьба' in queries_list:
        return 'свадьба'
    if 'недвижимость' in queries_list:
        return 'недвижимость'
    if 'жилье' in queries_list:
        return 'жилье'
    return 'автомобиль'
    
    
    
    
df['category_purpose'] = df['purpose'].apply(lemma)
print(df['category_purpose'])

0               жилье
1          автомобиль
2               жилье
3         образование
4             свадьба
             ...     
21520           жилье
21521      автомобиль
21522    недвижимость
21523      автомобиль
21524      автомобиль
Name: category_purpose, Length: 21525, dtype: object


### Вывод
1) провел лематизацию в столбце purpuse и сздал новый столбец category_purpose в котором вывел на основании лематизации столбца purpuse категории для удобства выведения категоризации данных и их сравнения

### Категоризация данных

In [453]:
def category_income(income):
    if income <= 50000:
        return 'низкая'
    if 50000 < income < 200000:
        return 'средняя'
    return 'высокая'

df['earnings'] = df['total_income'].apply(category_income)
print(df['earnings'])

0        высокая
1        средняя
2        средняя
3        высокая
4        средняя
          ...   
21520    высокая
21521    средняя
21522    средняя
21523    высокая
21524    средняя
Name: earnings, Length: 21525, dtype: object


In [454]:
df['earnings'].value_counts()

средняя    16086
высокая     5067
низкая       372
Name: earnings, dtype: int64

In [455]:
def category_children(famly):
    if famly == 0:
        return 'бездетная'
    if 1 <= famly <= 2:
        return 'среднедетная'
    return 'многодетная'

df['category children'] = df['children'].apply(category_children)
print(df['category children'])

0        среднедетная
1        среднедетная
2           бездетная
3         многодетная
4           бездетная
             ...     
21520    среднедетная
21521       бездетная
21522    среднедетная
21523     многодетная
21524    среднедетная
Name: category children, Length: 21525, dtype: object


### Вывод

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [456]:
df_pivot = df.pivot_table(index=['category children'], columns='debt', values='category_purpose', aggfunc='count')
df_pivot = df_pivot.fillna(0).astype('int').reset_index()
df_pivot['share_of_no_return'] = df_pivot[1] / (df_pivot[1] + df_pivot[0])

print(df_pivot.sort_values(by = 'share_of_no_return', ascending = False))

debt category children      0     1  share_of_no_return
2         среднедетная   6349   647            0.092481
1          многодетная    349    31            0.081579
0            бездетная  13086  1063            0.075129


### Вывод

зависимость есть. 
бездетные берут больше всего кредитов, а в многодетных меньше, но доля невозврата у среднедетной больше всего.
В то время как в бездетной меньше всего.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [457]:
df_pivot = df.pivot_table(index=['family_status'], columns='debt', values='category_purpose', aggfunc='count')
df_pivot = df_pivot.fillna(0).astype('int').reset_index()
df_pivot['share_of_no_return'] = df_pivot[1] / (df_pivot[1] + df_pivot[0])

print(df_pivot.sort_values(by = 'share_of_no_return', ascending = False))

debt          family_status      0    1  share_of_no_return
0     Не женат / не замужем   2539  274            0.097405
3          гражданский брак   3789  388            0.092890
4           женат / замужем  11449  931            0.075202
1                 в разводе   1110   85            0.071130
2            вдовец / вдова    897   63            0.065625


### Вывод

Зависимость есть. Больше всего кридитов берут 'женат / замужем', но доля невозврата у них средний и равен разведенным людям. Самая большая доля невозвратов у людей с статусом 'Не женат / не замужем'. меньше всего кредитов берут люди с статусом 'вдовец / вдова', доля невозврата у них меньше всего.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [458]:
df_pivot = df.pivot_table(index=['earnings'], columns='debt', values='category_purpose', aggfunc='count')
df_pivot = df_pivot.fillna(0).astype('int').reset_index()
df_pivot['share_of_no_return'] = df_pivot[1] / (df_pivot[1] + df_pivot[0])

print(df_pivot.sort_values(by = 'share_of_no_return', ascending = False))

debt earnings      0     1  share_of_no_return
2     средняя  14726  1360            0.084546
0     высокая   4709   358            0.070653
1      низкая    349    23            0.061828


### Вывод

зависимость есть. люди с средней заработной платой берут больше всего кредитов и доля невозврата у них больше всего. Меньше всего берут кредит люди с низкой заработной платой и процент невозврата у них меньше всего.

- Как разные цели кредита влияют на его возврат в срок?

In [459]:
df_pivot = df.pivot_table(index=['category_purpose'], columns='debt', values='total_income', aggfunc='count')
df_pivot = df_pivot.fillna(0).astype('int').reset_index()
df_pivot['share_of_no_return'] = df_pivot[1] / (df_pivot[1] + df_pivot[0])
df_pivot['share of loans in total'] = (df_pivot[1] + df_pivot[0]) / ((df_pivot[1].sum()) + (df_pivot[0].sum()))



print(df_pivot.sort_values(by = 'share_of_no_return', ascending = False))

debt category_purpose     0    1  share_of_no_return  share of loans in total
0          автомобиль  3912  403            0.093395                 0.200465
3         образование  3652  370            0.091994                 0.186852
4             свадьба  2162  186            0.079216                 0.109082
2        недвижимость  5893  474            0.074446                 0.295796
1               жилье  4165  308            0.068858                 0.207805


### Вывод

1)добавил столбец 'share of loans in total' где показал доляю кредитов от общего числа кредитов
2)зависимость есть. чем цель более важна для существования тем меньше доля невозврата.

In [460]:
import plotly.express as px

def pivot(data, index):
    pivot = data.pivot_table(index = index, values = 'debt', aggfunc = ['count', 'sum', 'mean']).reset_index()
    pivot.columns = [index, 'Кол-во клиентов', 'Кол-во должников', '% невозврата']
    display(pivot.sort_values('% невозврата', ascending = False))
    
    
    
    fig = px.bar(pivot, x = index, y = '% невозврата', color = '% невозврата', title = '% невозврата по ' + index)
    fig.show()
    

pivot(df, 'category_purpose')

,category_purpose,Кол-во клиентов,Кол-во должников,% невозврата
0,автомобиль,4315,403,0.093395
3,образование,4022,370,0.091994
4,свадьба,2348,186,0.079216
2,недвижимость,6367,474,0.074446
1,жилье,4473,308,0.068858


### Шаг 4. Общий вывод

1)люди с самым большой долей невозврата среднететные не находящиеся в браке с средней зароботной платой бирущие кредит на автомобиль
2)люди с самой маленькой долей невозврата бездетные вдовец/вдова с низкой зароботной платой бирущие кредит на жилье